In [1]:
#%run ann.py

Epoch 1/100
124/124 [==============================] - 1s 984us/step - loss: 0.6863 - accuracy: 0.7262
Epoch 2/100
124/124 [==============================] - 0s 769us/step - loss: 0.5030 - accuracy: 0.8254
Epoch 3/100
124/124 [==============================] - 0s 953us/step - loss: 0.3586 - accuracy: 0.82940s - loss: 0.3662 - accuracy: 0.
Epoch 4/100
124/124 [==============================] - 0s 686us/step - loss: 0.3300 - accuracy: 0.8338
Epoch 5/100
124/124 [==============================] - 0s 711us/step - loss: 0.3137 - accuracy: 0.8394
Epoch 6/100
124/124 [==============================] - 0s 720us/step - loss: 0.2966 - accuracy: 0.8478
Epoch 7/100
124/124 [==============================] - 0s 668us/step - loss: 0.2963 - accuracy: 0.8477
Epoch 8/100
124/124 [==============================] - 0s 658us/step - loss: 0.2843 - accuracy: 0.8525
Epoch 9/100
124/124 [==============================] - 0s 657us/step - loss: 0.2652 - accuracy: 0.8657
Epoch 10/100
124/124 [===================

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 28 18:37:30 2018

@author: alber
"""

# Librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math


###############################################################################

# Data Preparation

###############################################################################

# Dataset -> https://www.kaggle.com/ntnu-testimon/paysim1
#dataset = pd.read_csv('paysim.csv')
#dataset = dataset.sort_values(by='isFraud', ascending=False)
#dataset.reset_index(drop=True, inplace=True)
#d = dataset[dataset['isFraud']==1]
#dataset = dataset.iloc[0:len(d)*2]
#dataset.to_csv('paysim_reduced.csv')

dataset = pd.read_csv('paysim_reduced.csv')

# Data Preparation
df_aux = pd.get_dummies(dataset['type']).astype('int')
dataset.drop(['type', 'Unnamed: 0', 'nameOrig', 
              'nameDest', 'isFlaggedFraud', 'step',
              'newbalanceOrig', 'newbalanceDest'], axis=1, inplace=True)
dataset = dataset.join(df_aux)

In [4]:
dataset.head().T

,0,1,2,3,4
amount,850002.52,18887.22,20311.17,20311.17,176273.29
oldbalanceOrg,850002.52,18887.22,20311.17,20311.17,176273.29
oldbalanceDest,6510099.11,0.00,525.19,0.00,0.00
isFraud,1.00,1.00,1.00,1.00,1.00
CASH_IN,0.00,0.00,0.00,0.00,0.00
CASH_OUT,1.00,1.00,1.00,0.00,1.00
DEBIT,0.00,0.00,0.00,0.00,0.00
PAYMENT,0.00,0.00,0.00,0.00,0.00
TRANSFER,0.00,0.00,0.00,1.00,0.00


In [5]:
df_aux.head().T

,0,1,2,3,4
CASH_IN,0,0,0,0,0
CASH_OUT,1,1,1,0,1
DEBIT,0,0,0,0,0
PAYMENT,0,0,0,0,0
TRANSFER,0,0,0,1,0


In [3]:
X = dataset.loc[:, dataset.columns != 'isFraud'].values
y = dataset['isFraud'].values

# Train/Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [4]:
###############################################################################

# ANN Build

###############################################################################

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense


def create_nn(n_features, w_in, w_h1, n_var_out, optimizer, lr, momentum, decay):
    """
    Funcion para crear una NN para clasificacion binaria usando 2 HL
    
    """
    
    
    # Initialising the ANN
    model = Sequential()
    
    # First HL
    # [batch_size x n_features] x [n_features x w_in]
    model.add(Dense(units = w_in, input_dim = n_features, 
                    kernel_initializer = 'normal', 
                    activation = 'relu')) 
    # Second HL
    # [batch_size x w_in] x [w_in x w_h1]
    model.add(Dense(units = w_h1, input_dim = w_in, 
                    kernel_initializer = 'normal', 
                    activation = 'relu'))
    
    # Output Layer
    # [batch_size x w_h1] x [w_h1 x w_out]
    model.add(Dense(units = n_var_out, 
                    kernel_initializer = 'normal', 
                    activation = 'sigmoid')) 
    
    # Compile Model
    # Loss Function -> Cross Entropy (Binary)
    # Optimizer -> sgd, adam...
    if optimizer == 'sgd':
        keras.optimizers.SGD(lr=lr, momentum=momentum, decay=decay, nesterov=False)
        model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
    else:
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        
    return model


In [6]:
## fix random seed for reproducibility
#from tensorflow import set_random_seed
#from numpy.random import seed
#value = 7
#seed(value)
#set_random_seed(value)


# Parametros
n_features = np.shape(X_train)[1]
w_in = 12
w_h1 = 8
n_var_out = 1
batch_size = 100
nb_epochs = 100
optimizer = 'adam'
lr = 0.1
momentum = 0.01
decay = 0.0

# Create NN
model = create_nn(n_features, w_in, w_h1, n_var_out, optimizer, lr, momentum, decay)
    
# Fitting the ANN to the Training set
model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs)


Epoch 1/100
124/124 [==============================] - 1s 748us/step - loss: 0.6836 - accuracy: 0.7058
Epoch 2/100
124/124 [==============================] - 0s 738us/step - loss: 0.5128 - accuracy: 0.8079
Epoch 3/100
124/124 [==============================] - 0s 933us/step - loss: 0.3615 - accuracy: 0.8344
Epoch 4/100
124/124 [==============================] - 0s 997us/step - loss: 0.3263 - accuracy: 0.8385
Epoch 5/100
124/124 [==============================] - 0s 754us/step - loss: 0.3002 - accuracy: 0.8513
Epoch 6/100
124/124 [==============================] - 0s 641us/step - loss: 0.2936 - accuracy: 0.8548
Epoch 7/100
124/124 [==============================] - 0s 689us/step - loss: 0.2846 - accuracy: 0.8626
Epoch 8/100
124/124 [==============================] - 0s 681us/step - loss: 0.2698 - accuracy: 0.8699
Epoch 9/100
124/124 [==============================] - 0s 689us/step - loss: 0.2559 - accuracy: 0.8784
Epoch 10/100
124/124 [==============================] - 0s 689us/step - l

In [7]:
###############################################################################

# ANN Predictions

###############################################################################

# Predict
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)

# Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [9]:
print(cm)

[[1964  108]
 [  16 2019]]


In [10]:
y_pred

array([[0.],
       [0.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)